# Selenium 네이버 뉴스 댓글 스크래핑

## Selenium 및 웹 드라이버 설치

In [1]:
!pip install Selenium
!apt-get undate
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 2.7MB/s 
E: Invalid operation undate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 39 not upgraded.
Need to get 76.2 MB of archives.
After this operation, 255 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 84.0.4147.105-0ubuntu0.18.04.1 [1,118 kB]
Get:2 http://arc

## 라이브러리 import

In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## `news_scraping()`: 뉴스 스크래핑

In [54]:
def news_scraping(news_url,wd):
    press = wd.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[1]/a/img').get_attribute('title')
    title = wd.find_element_by_id('articleTitle').text
    datetime = wd.find_element_by_class_name('t11').text
    article = wd.find_element_by_id('articleBodyContents').text
    article = article.replace('// flash 오류를 우회하기 위한 함수 추가','')
    article = article.replace('function _flash_removeCallback() {}','')
    article = article.replace('\n','')
    article = article.replace('\t','')

    good = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[1]/a/span[2]').text
    warm = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[2]/a/span[2]').text
    sad = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[3]/a/span[2]').text
    angry = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[4]/a/span[2]').text
    want = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[5]/a/span[2]').text
    recom = wd.find_element_by_xpath('//*[@id="toMainContainer"]/a/em[2]').text

    print('뉴스',[title,press,datetime,article, good, warm, sad, angry, want,recom, news_url])

    return [title,press,datetime,article, good, warm, sad, angry, want,recom, news_url]



## `comments_scraping()`: 뉴스 댓글 스크래핑

In [55]:
def comments_scraping(news_url,wd):

    try:
        btn_view = wd.find_element_by_class_name('u_cbox_btn_view_comment').click()
        print('[댓글 더보기]', end = '')
        time.sleep(1)

        while True:
            wd.find_element_by_class_name('u_cbox_btn_more').click()
            print('[더보기]', end = '')
            time.sleep(1)

    except:
        pass

    btn_reply_list = wd.find_elements_by_class_name('u_cbox_btn_reply')
    for btn_reply in btn_reply_list:
        btn_reply.send_keys('\n')
        print('[답글]',end = '')
        time.sleep(1)

    print('[댓글스크래핑]')
    comments_idx = 0    #갯수 세주는 용도
    comments_df = pd.DataFrame(columns = ('contents','name','datetime','recommend','unrecommend','url'))

    comments = wd.find_elements_by_class_name('u_cbox_comment_box')
    for comment in comments:
        try:
            name = comment.find_element_by_class_name('u_cbox_nick').text
            date = comment.find_element_by_class_name('u_cbox_date').text
            contents = comment.find_element_by_class_name('u_cbox_contents').text
            recomm = comment.find_element_by_class_name('u_cbox_cnt_recomm').text
            unrecomm = comment.find_element_by_class_name('u_cbox_cnt_unrecomm').text   
            print(f'댓글 #{comments_idx+1}:',[contents,name,date,recomm,unrecomm,news_url])

            comments_df.loc[comments_idx] = [contents,name,date,recomm,unrecomm,news_url]
            comments_idx += 1
        except NoSuchElementException:
            print('[삭제되거나 부적절한 댓글]')
            continue    # 오류 걸르고 나면 계속 이어가야지

    return comments_df


## `scraping()`: 스크래핑 함수

In [67]:
def scraping(news_max=3):
    wd = webdriver.Chrome('chromedriver', options=chrome_options)
    wd.implicitly_wait(3)

    wd.execute_script('window.open("about:blank", "_blank");')  # 새창으로 텝이 하나 추가된다 가정하고
    tabs = wd.window_handles

    wd.switch_to.window(tabs[0])
    query = input('검색어 입력 : ')
    # search_url = 'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=' + query
    search_url = 'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=' + query
    wd.get(search_url)


    news_idx = 0
    news_df = pd.DataFrame(columns = ('title','press','datetime','article','good','warm','sad','angry','want','recommed','url'))
    comments_df = pd.DataFrame()


    while True:
        inline_list = wd.find_elements_by_class_name('txt_inline')
        for inline in inline_list:
            try:
                news_url = inline.find_element_by_class_name('_sp_each_url').get_attribute('href')
            except:
                continue

            wd.switch_to.window(tabs[1])    # 2번째 텝
            wd.get(news_url)

            news_df.loc[news_idx] = news_scraping(news_url,wd)
            news_idx += 1
#scraping
            df = comments_scraping(news_url,wd)    # df로 받아와야하니까
            comments_df = pd.concat([comments_df,df])

            if news_idx >= news_max:
                break
            
        if news_idx >= news_max:
            break

        try:
            wd.switch_to.window(tabs[0])
            wd.find_element_by_class_name('next').click()
            time.sleep(1)
        except:
            break

    wd.close()

    return news_df, comments_df

## 스크래핑 실행

In [68]:
news_df, comments_df = scraping()

검색어 입력 : 컴퓨터
뉴스 ["넥슨컴퓨터박물관, '게임을 게임하다' 온라인 전시관 오픈", 'ZDNet Korea', '2020.09.02. 오전 11:21', "(지디넷코리아=이도원 기자)넥슨컴퓨터박물관(관장 최윤아)은 '게임을 게임하다 / 인바이트 유(invite you)'의 온라인 전시관을 오픈했다고 2일 밝혔다.해당 전시관은 넥슨재단과 넥슨컴퓨터박물관, 넥슨 인텔리전스랩스는 지난 2019년 7월 18일부터 9월 1일까지 대한민국 온라인게임 25주년을 기념하기 위해 아트선재센터에서 마련했던 것을 1년만에 온라인으로 선보인 것이라고 박물관 측은 서렴ㅇ했다.전시관은 온라인게임의 특징인 참여와 성장을 모티브로 오프라인에 구현했던 전시를 다시 온라인 공간으로 옮겨 공개한다. 관람객은 로그인에서 시작해 로그아웃으로 끝나는 실제의 전시 관람 동선을 따라 ‘차원의 도서관’, ‘캠프파이어’, ‘로나와 판 & NPC’ 등 약 20개의 작품을 관람할 수 있으며, 대부분의 체험 요소 또한 그대로 경험할 수 있다.넥슨컴퓨터박물관 온라인게임 25주년 기획전시 '게임을 게임하다'.이는 전시 공간과 작품들을 사진, 영상 매체로 일방향적으로 기록하는 것을 넘어, 전시 관람이라는 인터랙티브한 체험 자체를 온라인 상에 아카이빙 하기 위함이다.또한 오프라인 전시와 마찬가지로 넥슨 ID와의 연동을 통해 넥슨 게임을 즐겼던 유저들은 각각 자신의 게임 플레이의 데이터를 영수증 형태로 시각화 한 아카이빙 자료를 다운로드 할 수 있다.최윤아 넥슨컴퓨터박물관 관장은 “본 전시는 온라인게임이 만들어지고 진행되는 과정을 재해석하고, 개개인의 시각으로 온라인게임의 역사를 해석해 볼 수 있는 전시였다”며 “세계적으로 유사한 사례를 찾아보기 어려운 온라인게임에 특성화 된 전시였던 만큼 관람 경험을 확장하고 보다 새로운 방식으로 아카이빙 하고자 했다”고 전했다.이도원 기자(leespot@zdnet.co.kr)▶ 지디넷코리아 '홈페이지'▶ 네이버 채널 구독하기[Copyrightⓒ메가뉴스 & ZDNet & CN

In [69]:
news_df

,title,press,datetime,article,good,warm,sad,angry,want,recommed,url
0,"넥슨컴퓨터박물관, '게임을 게임하다' 온라인 전시관 오픈",ZDNet Korea,2020.09.02. 오전 11:21,(지디넷코리아=이도원 기자)넥슨컴퓨터박물관(관장 최윤아)은 '게임을 게임하다 / 인...,0,0,0,0,0,,https://news.naver.com/main/read.nhn?mode=LSD&...
1,"중고컴퓨터 리뉴올PC, 이마트 일렉트로마트 킨텍스점 입점 할인행사 및 이벤트 진행",헤럴드경제,2020.09.02. 오전 10:30,[헤럴드경제] 중고컴퓨터 브랜드 리뉴올PC가 지난 8월7일 이마트 일렉트로마트 영등...,0,0,0,0,0,,https://news.naver.com/main/read.nhn?mode=LSD&...
2,[THE 사건/단독]“코로나로 영업정지 됐는데…” 문닫은 PC방서 컴퓨터·현금 털려,동아일보,2020.08.31. 오후 5:24,신종 코로나 바이러스(코로나19) 확산에 따른 ‘사회적 거리두기’ 2단계 시행으로 ...,0,0,23,103,2,23,https://news.naver.com/main/read.nhn?mode=LSD&...


In [70]:
comments_df

,contents,name,datetime,recommend,unrecommend,url
0,자영업자월세부담건물주가부담하고세를깍아주세요ㅠ강제로투자한가게나가지두못하는데월세관리비에...,ehdt****,2020.09.02. 08:07,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
1,건물주는 따박 따박 월세받고 자영업자들은 장사도 못하는데 월세까지 다내고ㅋㅋㅋ 좀 ...,kain****,2020.09.01. 05:12,3,1,https://news.naver.com/main/read.nhn?mode=LSD&...
2,안산에서 탁구친구라는 조그만 탁구장을 운영하고 있습니다. 현재 영업정지 중이구요. ...,rmrw****,2020.08.31. 19:34,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
3,광화문서 집회해도 pc방 잘못교회서 예배봐도 pc방 잘못도난 당해도 pc방 잘못정작...,good****,2020.08.31. 15:09,3,0,https://news.naver.com/main/read.nhn?mode=LSD&...
4,이런 범행 하지 맙시다... 다들 고생하고 계시고 다들 힘듭니다.. 모두들 서로서로...,sell****,2020.08.31. 14:52,1,0,https://news.naver.com/main/read.nhn?mode=LSD&...
5,훠훠훠 내알바 아니지요,lock****,2020.08.31. 13:30,1,6,https://news.naver.com/main/read.nhn?mode=LSD&...
6,진짜 피와 살도 모자라서 뼈까지 털어가네..,rich****,2020.08.31. 13:26,1,0,https://news.naver.com/main/read.nhn?mode=LSD&...
7,죽어라죽어라하는구나ㅡㅡ,ddal****,2020.08.31. 13:09,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
8,12일간 영업안했는데 현금23만은 금고에 보관한건 이해가 안되네 대형PC방인가보네.,xogm****,2020.08.31. 12:25,0,5,https://news.naver.com/main/read.nhn?mode=LSD&...
9,넌 원래 아무것도 이해를 못하는 놈이야 이제와서 머리통 굴려봐야 아무것도 이해를 못...,youn****,2020.08.31. 12:51,4,0,https://news.naver.com/main/read.nhn?mode=LSD&...
